In [1]:
import numpy as np

입력값과 임의의 타겟값의 지정

In [2]:
data = np.array(([1],[2],[3]))
data

array([[1],
       [2],
       [3]])

In [3]:
target = np.array(([0.5],[0.1]))
target

array([[0.5],
       [0.1]])

임의의 가중치 지정

In [4]:
w = []

In [5]:
w.append(np.array([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]]))

In [6]:
w.append(np.array([[0.7, 0.8], [0.9, 0.1]]))

In [7]:
b = []

In [8]:
b.append(np.array([[0.5]]))

In [9]:
b.append(np.array([[0.3]]))

In [10]:
# 비용 함수 클래스, 비용 함수의 종류에 따라 다른 클래스 내 함수를 사용한다.
class cost_function:
  # 예측값
  predict = []
  # 타겟값
  target = []
  # 비용 함수값
  error_cost = []

  # 오차 제곱합
  def errer_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    self.error_cost = np.sum(0.5*((predict - target)**2))
    return self.error_cost

  # 오차 제곱합 미분 함수
  def diff_error_squared_sum(self):
    return self.predict - self.target

In [11]:
# 활성화 함수
class activation_function:
  # 시그모이드 함수
  def sigmoid(self, x):
    return 1 / (1+np.e**-x)
  
  # 시그모이드 함수의 미분 함수
  def sigmoid_diff(self, x):
    return x * (1 - x)

In [12]:
class MLP:
  #입력값
  input_data = []
  #가중치
  weight = []
  #편향값
  bias = []
    
  # 각 층별 노드들의 입력
  node_input = []
  # 각 층별 노드들의 출력
  node_output = []

  # 순전파 연산 결과, 예상값
  predict = []
  # 타겟, 목푯값
  target = []

  # 각 층의 델타 값의 저장
  delta = []

  # 비용 함수
  cost = cost_function()

  # 활성화 함수
  activation = activation_function()

  # 가중치 업데이트 크기
  weight_update = []
  
  #순전파 계산산
  def forward_cal(self, input, w, b):
    # 입력값의 저장
    self.input_data = input
    
    #가중치와 편향 값의 지정
    self.weight = w
    self.bias = b

    #데이터 입력값이 입력층의 노드의 출력이 됨
    self.node_output.append(self.input_data)
        
    #3차원 배열 형태의 가중치의 층 갯수만큼 순전파 연산의 반복
    for i in range(len(self.weight)):
      #가중치와 노드 출력의 행렬곱연산, 편향값 덧셈
      hidden_input = w[i].T @ self.node_output[i] + b[i][0]
            
      #노드 입력과 활성화 함수 연산을 통한 노드 출력 계산
      output = self.activation.sigmoid(hidden_input)
            
      #노드 출력의 저장(델타 값의 계산을 위해)
      self.node_output.append(output)
      
    #최종 연산 결과, 예측값의 저장
    self.predict = output

    return output

  def cost_function(self, target):
    self.target = target
    return self.cost.errer_squared_sum(self.predict, target)

  #델타(노드 변화량에 대한 비용 함수의 변화량) 계산산
  def cal_delta(self):
    #출력층 노드의 변화량에 대한 오차 함수의 변화량 계산
    delta = (self.cost.diff_error_squared_sum() * self.activation.sigmoid_diff(self.predict))
    self.delta.append(delta)

    #이전층 노드의 delta 값 계산과 함께, 가중치 변화량에 대한 비용 함수의 변화량을 계산한다.
    for i in range(len(self.node_output) - 2, -1, -1):

      weight_update = (delta @ self.node_output[i].T).T
      self.weight_update.append(weight_update)
      delta = self.weight[i]@delta
      delta = delta * self.activation.sigmoid_diff(self.node_output[i])
      self.delta.append(delta)
  
  # 가중치 업데이트 크기 계산 함수
  def update_weight(self, learning_rate):
    # 거꾸로 계산된 가중치 업데이트 배열을 뒤집어준다.
    self.weight_update = self.weight_update[::-1]
    for i in range(len(self.weight)):
      weight = self.weight[i]
      weight_update = self.weight_update[i]

      self.weight[i] = weight - weight_update * learning_rate

  # 반복 학습을 위한 함수
  def forward_cal_iterations(self, iterations, learning_rate):
    self.node_output.append(self.input_data)
    
    # 지정한 반복 횟수만큼 학습 시행
    for i in range(iterations):
      # 순전파 계산 수행
      for j in range(len(self.weight)):
        hidden_input = self.weight[j].T @ self.node_output[j] + self.bias[j][0]

        output = self.activation.sigmoid(hidden_input)

        self.node_output.append(output)
      
      self.predict = output
      print(self.cost_function(self.target), "비용 함수값")
      self.cal_delta
      self.weight_update = self.weight_update[::-1]
      self.weight_update
      self.update_weight(learning_rate)





In [13]:
mlp = MLP()

In [14]:
result = mlp.forward_cal(data, w, b)
result

array([[0.86103399],
       [0.75879129]])

In [15]:
mlp.cost_function(target)

0.2821757530435978

In [16]:
mlp.cal_delta()

In [17]:
mlp.delta

[array([[0.04319933],
        [0.12057664]]), array([[0.00747633],
        [0.00174744]]), array([[ 0.        ],
        [-0.00588375],
        [-0.02871977]])]

In [18]:
mlp.weight

[array([[0.1, 0.2],
        [0.3, 0.4],
        [0.5, 0.6]]), array([[0.7, 0.8],
        [0.9, 0.1]])]

In [19]:
mlp.weight_update

[array([[0.04047892, 0.11298352],
        [0.04166268, 0.11628758]]), array([[0.00747633, 0.00174744],
        [0.01495265, 0.00349488],
        [0.02242898, 0.00524232]])]

In [20]:
mlp.delta

[array([[0.04319933],
        [0.12057664]]), array([[0.00747633],
        [0.00174744]]), array([[ 0.        ],
        [-0.00588375],
        [-0.02871977]])]

In [21]:
mlp.update_weight(0.01)

In [22]:
mlp.weight

[array([[0.09992524, 0.19998253],
        [0.29985047, 0.39996505],
        [0.49977571, 0.59994758]]), array([[0.69959521, 0.79887016],
        [0.89958337, 0.09883712]])]

In [23]:
mlp.forward_cal_iterations(1000, 0.01)

0.2818790553005689 비용 함수값
0.28158221051900706 비용 함수값
0.28128521937260265 비용 함수값
0.28098808253705665 비용 함수값
0.28069080069007774 비용 함수값
0.28039337451137913 비용 함수값
0.2800958046826753 비용 함수값
0.27979809188767857 비용 함수값
0.27950023681209507 비용 함수값
0.2792022401436223 비용 함수값
0.27890410257194553 비용 함수값
0.27860582478873314 비용 함수값
0.27830740748763433 비용 함수값
0.2780088513642749 비용 함수값
0.27771015711625363 비용 함수값
0.2774113254431389 비용 함수값
0.27711235704646453 비용 함수값
0.2768132526297263 비용 함수값
0.2765140128983779 비용 함수값
0.2762146385598271 비용 함수값
0.2759151303234323 비용 함수값
0.2756154889004978 비용 함수값
0.27531571500427043 비용 함수값
0.2750158093499355 비용 함수값
0.27471577265461206 비용 함수값
0.27441560563734957 비용 함수값
0.27411530901912323 비용 함수값
0.27381488352283045 비용 함수값
0.27351432987328556 비용 함수값
0.2732136487972165 비용 함수값
0.2729128410232599 비용 함수값
0.2726119072819569 비용 함수값
0.2723108483057488 비용 함수값
0.2720096648289729 비용 함수값
0.27170835758785694 비용 함수값
0.27140692732051575 비용 함수값
0.27110537476694624 비용 함수값
0.270803700669022